<a href="https://colab.research.google.com/github/jairft12-bot/quejas/blob/main/jair_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# --- Cargar datos ---
df = pd.read_excel("/content/RECLAMOS Y SOLICITUDES PERÚ.1.xlsx", sheet_name="DATA CORREO")
df["FECHA DEL INCIDENTE"] = pd.to_datetime(df["FECHA DEL INCIDENTE"], errors="coerce")
df["FECHA DE RESPUESTA"] = pd.to_datetime(df["FECHA DE RESPUESTA"], errors="coerce")
df["DIAS_RESOLUCION"] = (df["FECHA DE RESPUESTA"] - df["FECHA DEL INCIDENTE"]).dt.days

# --- Función para dibujar dashboard filtrado ---
def dashboard_filtrado(asegurador, sede, estado, especialidad, tipo_contrato, inicio, fin):
    df_filtrado = df.copy()

    # Aplicar filtros
    if asegurador != "Todos":
        df_filtrado = df_filtrado[df_filtrado["ASEGURADOR"] == asegurador]
    if sede != "Todos":
        df_filtrado = df_filtrado[df_filtrado["SEDE"] == sede]
    if estado != "Todos":
        df_filtrado = df_filtrado[df_filtrado["ESTADO FINAL"] == estado]
    if especialidad != "Todos":
        df_filtrado = df_filtrado[df_filtrado["ESPECIALIDAD"] == especialidad]
    if tipo_contrato != "Todos":
        df_filtrado = df_filtrado[df_filtrado["TIPO CONTRATO"] == tipo_contrato]
    if inicio:
        df_filtrado = df_filtrado[df_filtrado["FECHA DEL INCIDENTE"] >= pd.to_datetime(inicio)]
    if fin:
        df_filtrado = df_filtrado[df_filtrado["FECHA DEL INCIDENTE"] <= pd.to_datetime(fin)]

    # Check if the filtered dataframe is empty
    if df_filtrado.empty:
        print("No data matches the selected filters.")
        return

    # --- KPIs ---
    total_reclamos = len(df_filtrado)
    resueltos_4_dias = (df_filtrado["DIAS_RESOLUCION"] < 4).sum()
    promedio_dias = df_filtrado["DIAS_RESOLUCION"].mean()

    print(f"Total reclamos: {total_reclamos}")
    print(f"% resueltos en <4 días: {resueltos_4_dias/total_reclamos*100:.1f}%")
    print(f"Tiempo promedio de resolución: {promedio_dias:.1f} días\n")

    # --- Grupos de días ---
    grupos = {
        "Menos de 4 días": (df_filtrado["DIAS_RESOLUCION"] < 4).sum(),
        "De 4 a 15 días": ((df_filtrado["DIAS_RESOLUCION"] >= 4) & (df_filtrado["DIAS_RESOLUCION"] <= 15)).sum(),
        "Más de 15 días": (df_filtrado["DIAS_RESOLUCION"] > 15).sum()
    }
    total = sum(grupos.values())

    # --- Top motivos ---
    motivos = df_filtrado["MOTIVO"].value_counts().head(10)

    # --- Top especialidades ---
    especialidades_top = df_filtrado["ESPECIALIDAD"].value_counts().head(10)


    # --- Evolución mensual ---
    reclamos_mensuales = df_filtrado.groupby(df_filtrado["FECHA DEL INCIDENTE"].dt.to_period("M")).size()

    # --- Estados finales ---
    estados = df_filtrado["ESTADO FINAL"].value_counts()

    # --- Dibujar dashboard ---
    plt.figure(figsize=(18,18)) # Increased figure size for better spacing

    # Torta
    plt.subplot(3,2,1) # Changed subplot grid to 3x2
    if total > 0: # Check if there are any values to plot
        def formato_etiqueta(val):
            cantidad = int(round(val*total/100))
            return f"{cantidad} ({val:.1f}%)"
        plt.pie(grupos.values(), labels=grupos.keys(), autopct=formato_etiqueta,
                startangle=90, colors=["#66c2a5","#8da0cb","#fc8d62"], wedgeprops={'edgecolor':'black'})
        plt.title("Resolución por grupo de días")
    else:
        plt.text(0.5, 0.5, "No data to display", horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.title("Resolución por grupo de días")


    # Top 10 motivos
    plt.subplot(3,2,2) # Changed subplot grid to 3x2
    if not motivos.empty: # Check if there are any values to plot
        plt.barh(motivos.index, motivos.values, color="skyblue", edgecolor="black")
        plt.xlabel("Cantidad de reclamos")
        plt.title("Top 10 motivos de reclamos")
        for i, v in enumerate(motivos.values):
            plt.text(v+0.5, i, str(v), va="center")
    else:
        plt.text(0.5, 0.5, "No data to display", horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.title("Top 10 motivos de reclamos")

    # Top 10 especialidades
    plt.subplot(3,2,3) # Added new subplot for top specialties
    if not especialidades_top.empty: # Check if there are any values to plot
        plt.barh(especialidades_top.index, especialidades_top.values, color="lightgreen", edgecolor="black")
        plt.xlabel("Cantidad de reclamos")
        plt.title("Top 10 especialidades con más reclamos")
        for i, v in enumerate(especialidades_top.values):
            plt.text(v+0.10, i, str(v), va="center")
    else:
        plt.text(0.10, 0.10, "No data to display", horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.title("Top 10 especialidades con más reclamos")


    # Evolución mensual
    plt.subplot(3,2,4) # Changed subplot grid to 3x2
    if not reclamos_mensuales.empty: # Check if there are any values to plot
        plt.plot(reclamos_mensuales.index.astype(str), reclamos_mensuales.values,
                 marker="o", color="seagreen")
        plt.xticks(rotation=45)
        plt.ylabel("Cantidad")
        plt.title("Evolución mensual de reclamos")
        for x, y in zip(reclamos_mensuales.index.astype(str), reclamos_mensuales.values):
            plt.text(x, y+0.5, str(y), ha="center", fontsize=9)
    else:
        plt.text(0.5, 0.5, "No data to display", horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.title("Evolución mensual de reclamos")


    # Estados finales
    plt.subplot(3,2,5) # Changed subplot grid to 3x2
    if not estados.empty: # Check if there are any values to plot
        bars = plt.bar(estados.index, estados.values, color="lightcoral", edgecolor="black")
        plt.xticks(rotation=45)
        plt.ylabel("Cantidad")
        plt.title("Reclamos por estado final")
        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x()+bar.get_width()/2, yval+0.5, str(yval), ha="center")
    else:
        plt.text(0.5, 0.5, "No data to display", horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.title("Reclamos por estado final")

    plt.subplots_adjust(hspace=0.02, wspace=4.5) # Increased horizontal spacing
    plt.tight_layout()
    plt.show()

# --- Widgets / filtros ---
interact(
    dashboard_filtrado,
    asegurador=["Todos"] + df["ASEGURADOR"].dropna().unique().tolist(),
    sede=["Todos"] + df["SEDE"].dropna().unique().tolist(),
    estado=["Todos"] + df["ESTADO FINAL"].dropna().unique().tolist(),
    especialidad=["Todos"] + df["ESPECIALIDAD"].dropna().unique().tolist(),
    tipo_contrato=["Todos"] + df["TIPO CONTRATO"].dropna().unique().tolist(),
    inicio=widgets.DatePicker(description="Fecha inicio", value=None),
    fin=widgets.DatePicker(description="Fecha fin", value=None)
)

interactive(children=(Dropdown(description='asegurador', options=('Todos', 'PERÚ'), value='Todos'), Dropdown(d…

<function __main__.dashboard_filtrado(asegurador, sede, estado, especialidad, tipo_contrato, inicio, fin)>

BPM (Gestión de Procesos de Negocio)

**texto en negrita**